# 快速入门 Assistants API 

Assistants API 允许您在自己的应用程序中构建人工智能助手。一个助手有其指令，并可以利用模型、工具和知识来回应用户查询。

Assistants API 目前支持三种类型的工具：
- 代码解释器 Code Interpreter
- 检索 Retrieval
- 函数调用 Function calling

使用 [Playground](https://platform.openai.com/playground?mode=assistant) 可以在线探索和测试 Assistants API 功能。

本入门指南将指导您完成创建和运行使用代码解释器的助手的关键步骤，以下是使用 Assistants API 标准流程：
1. 通过定义其自定义指令并选择 LLM 来创建一个助手(Assistant)。如果有需求，可以添加文件并启用诸如代码解释器、检索和函数调用等工具。
2. 当用户开始对话时，创建一个线程(Thread)。
3. 当用户提问时，向线程添加消息(Messages)。
4. 通过调用模型和工具在线程上运行助手以生成响应。

![assistans](images/diagram-assistant.png)

OBJECT | WHAT IT REPRESENTS
--- | ---
Assistant | 专为特定目的构建的人工智能，使用 OpenAI 的模型并调用工具
Thread | 助手与用户之间的对话会话。线程存储消息，并自动处理截断，以将内容适应模型的上下文。
Message | 由助手或用户创建的消息。消息可以包括文本、图片和其他文件。消息以列表形式存储在线程上。
Run | 在线程上对一个助手的调用。助手利用其配置和线程的消息执行任务，通过调用模型和工具。作为运行的一部分，助手会将消息追加到线程中。
Run Step | 助手在运行中采取的详细步骤列表。助手可以在其运行期间调用工具或创建消息。检查运行步骤可以让您深入了解助手如何得出最终结果。



## 使用 Assistants 开发数学辅导老师

在这个示例中，我们正在创建一个数学辅导助手，并启用了代码解释器工具。

### 第一步：创建助手

In [ ]:
! pip install -U openai

In [1]:
import openai  # 导入 openai 库

# 从环境变量 OPENAI_API_KEY 中获取 API 密钥
client = openai.OpenAI()

# 创建一个名为 "Math Tutor" 的助手，它是一个个人数学辅导老师。这个助手能够编写并运行代码来解答数学问题。
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],  # 使用工具：代码解释器
    model="gpt-4o",  # 使用模型： GPT-4o
)

### 第二步：创建线程

一个线程代表用户和一个或多个助手之间的对话。

In [2]:
# 创建一个交流线程
thread = client.beta.threads.create()

### 第三步：往线程添加消息

用户或APP创建的消息内容将作为消息对象（Message Object）添加到线程中。 

消息可以包含文本和文件，向线程添加的消息数量没有限制 - OpenAI 会智能地截断任何不适合模型上下文窗口的内容。

In [3]:
# 在该线程中创建一条用户消息，并提交一个数学问题：“我需要解方程 `3x + 11 = 14`。你能帮忙吗？”
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)

### 第四步：调用助手

一旦所有用户消息都添加到了线程中，你可以使用任何助手运行该线程。

创建一个运行会使用与助手相关的模型和工具来生成响应。这些响应将作为助手消息添加到线程中。

In [4]:
# 创建并等待执行流完成，用于处理该线程中的交互和问题解答
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Jane Doe. The user has a premium account.",  # 以 Jane Doe 称呼用户，并且用户拥有高级账户
)

print("Run completed with status: " + run.status)  # 打印执行流的完成状态

# 如果执行流状态为 "completed"（已完成），则获取并打印所有消息
if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=thread.id)

    print("\nMessages:\n")
    for message in messages:
        assert message.content[0].type == "text"
        print(f"Role: {message.role.capitalize()}")  # 角色名称首字母大写
        print("Message:")
        print(message.content[0].text.value + "\n")  # 每条消息后添加空行以增加可读性

Run completed with status: completed

Messages:

Role: Assistant
Message:
Of course, Jane Doe! To solve the equation \(3x + 11 = 14\), we need to isolate \(x\). Here are the steps:

1. Subtract 11 from both sides of the equation:
\[ 3x + 11 - 11 = 14 - 11 \]
\[ 3x = 3 \]

2. Divide both sides by 3:
\[ \frac{3x}{3} = \frac{3}{3} \]
\[ x = 1 \]

So, the solution to the equation \(3x + 11 = 14\) is \(x = 1\).

Would you like me to verify this solution?

Role: User
Message:
I need to solve the equation `3x + 11 = 14`. Can you help me?



### 通过 Assistant ID 删除指定助手

In [5]:
# 删除创建的助手
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_N4GlswRFWvHXoPffhbTLQYnh', deleted=True, object='assistant.deleted')

### 使用流式输出实现数学辅导老师

In [6]:
# 官方案例
from typing_extensions import override
from openai import AssistantEventHandler
import openai

# 从环境变量 OPENAI_API_KEY 中获取 API 密钥
client = openai.OpenAI()

# 创建一个名为 "Math Tutor" 的助手，它是一个个人数学辅导老师。这个助手能够编写并运行代码来解答数学问题。
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],  # 工具包括代码解释器
    model="gpt-4o-mini",  # 使用的模型是 GPT-4
)

# 创建一个交流线程
thread = client.beta.threads.create()

# 在该线程中创建一条消息，表示用户角色，并提交一个数学问题：“我需要解方程 `3x + 11 = 14`。你能帮忙吗？”
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)

print("starting run stream")  # 打印开始执行流的消息

# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 
# Then, we use the `stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.
 
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


# 删除创建的助手
client.beta.assistants.delete(assistant.id)

starting run stream

assistant > code_interpreter

import sympy as sp

# Define the variable
x = sp.symbols('x')

# Define the equation
equation = sp.Eq(3*x + 11, 14)

# Solve the equation
solution = sp.solve(equation, x)
solution
assistant > The solution to the equation \( 3x + 11 = 14 \) is \( x = 1 \).

AssistantDeleted(id='asst_m2l5WmP2WaBgZthhC2jW6Rps', deleted=True, object='assistant.deleted')

## 开发 Python 代码小助手

In [7]:
import openai  # 导入 openai 库
import time

# 从环境变量 OPENAI_API_KEY 中获取 API 密钥
client = openai.OpenAI()

# 创建一个名为 "Python Master" 的助手，它能根据需求生成可以运行的 Python 代码
assistant_python = client.beta.assistants.create(
    name="Python Master",
    instructions="You are a Python Expert. Generate runnable Python code according to messages.",
    tools=[{"type": "code_interpreter"}],  # 使用工具：代码解释器
    model="gpt-4o-mini",  # 使用模型： GPT-4
)

# 创建一个交流线程
thread_python = client.beta.threads.create()

In [9]:
assistant_python.id

'asst_efluGKh8WLe7ipLEChbHIf0K'

In [10]:
thread_python.id

'thread_wXeuAZQgd9rXKnDupXjrrdKx'

In [11]:
# 在该线程中创建一条信息
message = client.beta.threads.messages.create(
    thread_id=thread_python.id,
    role="user",
    content="快速排序咋个写？",
)

In [12]:
message.id

'msg_S8EWF20GN4SJhEdgZiaAkXeE'

In [13]:
# 创建并等待执行流完成，用于处理该线程中的交互和问题解答
# 方式一：create_and_poll创建并轮询方式
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread_python.id,
    assistant_id=assistant_python.id,
)

print("Run completed with status: " + run.status)  # 打印执行流的完成状态

Run completed with status: completed


In [14]:
run.id

'run_C5xEf8hHkQDAul3biQxMcsBz'

In [15]:
# 方式二：create 和 retrieve方式，判断run状态为（初始为queued）
run2 = client.beta.threads.runs.create(
    thread_id=thread_python.id,
    assistant_id=assistant_python.id,
)

print("Run初始状态 " + run2.status)  # 打印执行流的完成状态

Run初始状态 queued


In [16]:
run2.id

'run_KEGRNEPQzt6EM6vmRyrql68g'

In [17]:
while run2.status == "queued" or run.status == "in_progress":
    run2 = client.beta.threads.runs.retrieve(
        thread_id=thread_python.id,
        run_id=run2.id
    )
    time.sleep(1)
print("Run2 completed with status: " + run2.status)  # 打印执行流的完成状态

Run2 completed with status: completed


In [18]:
# 如果执行流状态为 "completed"（已完成），则获取并打印所有消息
if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=thread_python.id)

    print("\nMessages:\n")
    for message in messages:
        assert message.content[0].type == "text"
        print(f"Role: {message.role.capitalize()}")  # 角色名称首字母大写
        print("Message:")
        print(message.content[0].text.value + "\n")  # 每条消息后添加空行以增加可读性


Messages:

Role: Assistant
Message:
经过快速排序，数组 `[7, 2, 1, 6, 8, 5, 3, 4]` 已成功排序为 `[1, 2, 3, 4, 5, 6, 7, 8]`。如果你有其他问题或需要进一步的帮助，请告诉我！

Role: Assistant
Message:
快速排序是一种高效的排序算法，采用分治法（divide and conquer）策略来对数据进行排序。它的基本过程是：

1. 从数组中选择一个元素作为"枢轴"（pivot）。
2. 将数组分为两部分：一部分包含所有小于枢轴的元素，另一部分包含所有大于或等于枢轴的元素。
3. 对这两部分递归地进行快速排序。
4. 将所有部分合并成一个排序好的数组。

下面是快速排序的 Python 实现：

```python
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]  # 选择中间元素作为枢轴
    left = [x for x in arr if x < pivot]   # 小于枢轴的元素
    middle = [x for x in arr if x == pivot]  # 等于枢轴的元素
    right = [x for x in arr if x > pivot]  # 大于枢轴的元素
    return quick_sort(left) + middle + quick_sort(right)  # 递归排序并合并结果

# 测试代码
arr = [7, 2, 1, 6, 8, 5, 3, 4]
sorted_arr = quick_sort(arr)
print(sorted_arr)
```

你可以运行这段代码来查看快速排序的效果。

Role: User
Message:
快速排序咋个写？



In [19]:
# 在该线程中创建一条信息
message = client.beta.threads.messages.create(
    thread_id=thread_python.id,
    role="user",
    content="红黑树呢？",
)

In [20]:
# 创建并等待执行流完成，用于处理该线程中的交互和问题解答
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread_python.id,
    assistant_id=assistant_python.id,
)

print("Run completed with status: " + run.status)  # 打印执行流的完成状态

Run completed with status: completed


In [21]:
# 如果执行流状态为 "completed"（已完成），则获取并打印所有消息
if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=thread_python.id)

    print("\nMessages:\n")
    for message in messages:
        assert message.content[0].type == "text"
        print(f"Role: {message.role.capitalize()}")  # 角色名称首字母大写
        print("Message:")
        print(message.content[0].text.value + "\n")  # 每条消息后添加空行以增加可读性


Messages:

Role: Assistant
Message:
红黑树是一种自平衡的二叉搜索树，每个节点都有一个颜色（红色或黑色），并且必须满足以下性质：

1. 节点是红色或黑色。
2. 根节点是黑色。
3. 每个叶子节点（Nil或空节点）是黑色。
4. 如果一个节点是红色，则它的两个子节点必须是黑色（即，红色节点不能连续出现）。
5. 从任何节点到其每个叶子节点的路径都包含相同数量的黑色节点。

红黑树的这些性质保证了树的高度在 \(O(\log n)\)，从而确保了基本的操作（如插入、删除和查找）在对数时间内完成。

下面是一个简单的红黑树的 Python 实现，只包括插入操作。为了简化代码，这个实现只展示如何维持红黑树的性质，而不涉及迭代器和删除操作。

```python
class Node:
    def __init__(self, data, color='red'):
        self.data = data
        self.color = color
        self.left = None
        self.right = None
        self.parent = None

class RedBlackTree:
    def __init__(self):
        self.NIL_LEAF = Node(data=None, color='black')  # Nil叶节点
        self.root = self.NIL_LEAF

    def insert(self, data):
        new_node = Node(data)
        new_node.left = self.NIL_LEAF
        new_node.right = self.NIL_LEAF
        
        # 插入步骤
        parent = None
        current = self.root
        
        while current != self.NIL_LEAF:
            parent = current
            if new_node.data < c

In [22]:
# 删除创建的助手
client.beta.assistants.delete(assistant_python.id)

AssistantDeleted(id='asst_efluGKh8WLe7ipLEChbHIf0K', deleted=True, object='assistant.deleted')

## Homework: 将数学辅导老师和Python小助手的代码重构，实现类似 Playground 的对话输入体验